In [1]:
import tensorflow as tf
import keras_tuner as kt
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [3]:
iris = datasets.load_iris()
X = iris.data  # Features
y = iris.target.reshape(-1, 1)  # Targets reshaped for one-hot encoding

# One-hot encode the target labels
encoder = OneHotEncoder(sparse_output=False)
y_encoded = encoder.fit_transform(y)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Normalize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [5]:
def build_model(hp):
    model = tf.keras.Sequential()
    
    # First hidden layer with tunable number of neurons
    model.add(tf.keras.layers.Dense(hp.Int('units1', min_value=8, max_value=64, step=8), 
                                    activation=hp.Choice('activation1', ['relu', 'tanh']),
                                    input_shape=(4,)))
    
    # Second hidden layer with tunable neurons
    model.add(tf.keras.layers.Dense(hp.Int('units2', min_value=8, max_value=64, step=8),
                                    activation=hp.Choice('activation2', ['relu', 'tanh'])))
    
    # Output layer
    model.add(tf.keras.layers.Dense(3, activation='softmax'))
    
    # Compile the model
    model.compile(optimizer=tf.keras.optimizers.Adam(hp.Choice('learning_rate', [0.001, 0.01, 0.1])),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [6]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,  # Number of different models to try
    executions_per_trial=1,  # Number of times each model is trained
    directory='kt_iris',  
    project_name='iris_tuning'
)

C:\ProgramData\anaconda3\envs\tf_env\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
# Perform the search
tuner.search(X_train, y_train, epochs=50, validation_data=(X_test, y_test))

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

Trial 10 Complete [00h 00m 30s]
val_accuracy: 1.0

Best val_accuracy So Far: 1.0
Total elapsed time: 00h 04m 56s
